In [171]:
import pandas as pd
import numpy as np
chunk = pd.read_csv("./Anime dataset 2023/final_animedataset.csv", chunksize=1000000)

for df in chunk:
    break
df.head(5)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0,karthiga,21,9,2255153,Female,One Piece,TV,Manga,8.54,423868,91.0,35,"Action, Adventure, Comedy, Super Power, Drama,..."
1,karthiga,59,7,2255153,Female,Chobits,TV,Manga,7.53,175388,1546.0,188,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen"
2,karthiga,74,7,2255153,Female,Gakuen Alice,TV,Manga,7.77,33244,941.0,1291,"Comedy, School, Shoujo, Super Power"
3,karthiga,120,7,2255153,Female,Fruits Basket,TV,Manga,7.77,167968,939.0,222,"Slice of Life, Comedy, Drama, Romance, Fantasy..."
4,karthiga,178,7,2255153,Female,Ultra Maniac,TV,Manga,7.26,9663,2594.0,2490,"Magic, Comedy, Romance, School, Shoujo"


In [181]:
print(df.shape)
genre = df['genre'].str.split(', ')
unique_genre = list({x for l in genre.dropna() for x in l})
df1 = pd.DataFrame(df)
for i in unique_genre:
    df1[i] = genre.map(lambda x: i in x, na_action="ignore")
print(df1.head(5))

(1000000, 13)
   username  anime_id  my_score  user_id  gender          title type source  \
0  karthiga        21         9  2255153  Female      One Piece   TV  Manga   
1  karthiga        59         7  2255153  Female        Chobits   TV  Manga   
2  karthiga        74         7  2255153  Female   Gakuen Alice   TV  Manga   
3  karthiga       120         7  2255153  Female  Fruits Basket   TV  Manga   
4  karthiga       178         7  2255153  Female   Ultra Maniac   TV  Manga   

   score  scored_by  ...  Music   Kids  Harem Military  Mecha Adventure  \
0   8.54     423868  ...  False  False  False    False  False      True   
1   7.53     175388  ...  False  False  False    False  False     False   
2   7.77      33244  ...  False  False  False    False  False     False   
3   7.77     167968  ...  False  False  False    False  False     False   
4   7.26       9663  ...  False  False  False    False  False     False   

   Ecchi Mystery   Cars Parody  
0  False   False  False  Fa

In [199]:
unique_genre

['Shounen Ai',
 'Comedy',
 'Slice of Life',
 'Dementia',
 'Historical',
 'Samurai',
 'Shounen',
 'Space',
 'Sports',
 'Fantasy',
 'Romance',
 'Game',
 'Martial Arts',
 'Magic',
 'Psychological',
 'Demons',
 'Josei',
 'Vampire',
 'Horror',
 'Shoujo',
 'Shoujo Ai',
 'Seinen',
 'Yuri',
 'Sci-Fi',
 'Supernatural',
 'Super Power',
 'Drama',
 'School',
 'Police',
 'Hentai',
 'Thriller',
 'Yaoi',
 'Action',
 'Music',
 'Kids',
 'Harem',
 'Military',
 'Mecha',
 'Adventure',
 'Ecchi',
 'Mystery',
 'Cars',
 'Parody']

In [ ]:
#Check if there are mismatches among the same anime. Takes forever to run.(7700k 7:40 min)
#for title in df1["title"].unique():
#    genre_val = df1[df1["title"]==title][unique_genre].sum()
#    if (np.unique(genre_val).size > 2):
#        print("Theres genre mismatch for title", title)

In [200]:
sex = (df1["Harem"]==True)#(df1["Hentai"]==True) | (df1["Ecchi"]==True) | (df1["Harem"]==True) | (df1["Yaoi"]==True)
print(df1.loc[sex & (df1["gender"]=="Male")]["my_score"].mean())
print(df1.loc[sex & (df1["gender"]=="Male")]["my_score"].std())
print(df1.loc[sex & (df1["gender"]=="Female")]["my_score"].mean())
print(df1.loc[sex & (df1["gender"]=="Female")]["my_score"].std())

4.725298798363455
3.636676936605658
3.7706493709463156
3.7006823283698034


In [201]:
best = ("", 0)
for title in df1.loc[sex]["title"].unique():
    score = df1.loc[sex & (df1["title"]==title)]["score"].mean()
    if score > best[1]:
        best = (title, score)
    print(title, score)
print("Best sex", best)

Ouran Koukou Host Club 8.340000000000002
Kiniro no Corda: Primo Passo 7.630000000000001
Gakuen Heaven 6.74
School Days 6.019999999999997
Sekirei 7.319999999999999
White Album 6.680000000000001
ChäoS;HEAd 6.519999999999996
Kiss x Sis 7.19
Sora no Otoshimono 7.569999999999999
White Album 2nd Season 6.980000000000003
Ichiban Ushiro no Daimaou 7.139999999999999
Kami nomi zo Shiru Sekai 7.879999999999998
Kore wa Zombie Desu ka? 7.5900000000000025
Oniichan no Koto nanka Zenzen Suki ja Nai n da kara ne!! 6.269999999999999
Kami nomi zo Shiru Sekai II 8.069999999999999
Uta no☆Prince-sama♪ Maji Love 1000% 7.299999999999998
Boku wa Tomodachi ga Sukunai 7.47
High School DxD 7.620000000000001
Uta no☆Prince-sama♪ Maji Love 2000% 7.46
Oniichan dakedo Ai sae Areba Kankeinai yo ne! 6.71
Boku wa Tomodachi ga Sukunai Next 7.590000000000002
Kami nomi zo Shiru Sekai: Tenri-hen 7.8100000000000005
Hentai Ouji to Warawanai Neko. 7.39
Date A Live 7.4700000000000015
Brothers Conflict 6.51
Kami nomi zo Shiru Sek

In [193]:
pd.set_option('display.max_columns', None)
df1.loc[df1["title"]==best[0]].head(1)

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre,Shounen Ai,Comedy,Slice of Life,Dementia,Historical,Samurai,Shounen,Space,Sports,Fantasy,Romance,Game,Martial Arts,Magic,Psychological,Demons,Josei,Vampire,Horror,Shoujo,Shoujo Ai,Seinen,Yuri,Sci-Fi,Supernatural,Super Power,Drama,School,Police,Hentai,Thriller,Yaoi,Action,Music,Kids,Harem,Military,Mecha,Adventure,Ecchi,Mystery,Cars,Parody
5792,MistButterfly,33674,8,2485327,Female,No Game No Life: Zero,Movie,Light novel,8.74,78743,40.0,320,"Comedy, Ecchi, Fantasy, Game, Supernatural",False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [196]:

best_dementia = ("", 0)
for title in df1.loc[df1["Dementia"]==True]["title"].unique():
    score = df1.loc[df1["title"]==title]["score"].mean()
    if score > best_dementia[1]:
        best_dementia = (title, score)
    print(title, score)
print("Best dementia", best_dementia)

Neon Genesis Evangelion 8.319999999999999
Neon Genesis Evangelion: The End of Evangelion 8.480000000000004
FLCL 8.069999999999997
Mousou Dairinin 7.730000000000001
Serial Experiments Lain 8.0
Boogiepop wa Warawanai: Boogiepop Phantom 7.21
Perfect Blue 8.319999999999999
Nekojiru-sou 7.419999999999999
Mind Game 7.85
Tenshi no Tamago 7.629999999999998
Higurashi no Naku Koro ni 8.099999999999998
Paprika 8.120000000000001
Koroshiya 1 The Animation: Episode 0 5.860000000000001
Genius Party 7.340000000000002
Mawaru Penguindrum 8.03
Yami Shibai 7.179999999999999
Yami Shibai 2 6.55
Ousama Game The Animation 5.18
Kaze no Na wa Amnesia 6.529999999999999
Shoujo Kakumei Utena: Adolescence Mokushiroku 7.6
Ryokunohara Meikyuu 5.19
Kanashimi no Belladonna 7.2
Connected 6.120000000000001
Ai 4.4
Genius Party Beyond 7.3500000000000005
3-tsu no Kumo 4.96
Panty & Stocking in Sanitarybox 6.5
Utsu Musume Sayuri 2.08
00:08 5.02
Tamala 2010: A Punk Cat in Space 6.66
Inaka Isha 6.799999999999995
Kikumana 5.94
T